In [18]:
# Quelle est l'influence combinée des facteurs démographiques, économiques, de sécurité et sociaux sur les résultats des élections présidentielles de 2022 en France, par code postal?

In [19]:
# Feature Engineering

# Ajout d'une tranche d'age dans la démographie
# Faire le ratio entre le niveau de vie et le niveau de chomage
# Ajouter le taux de criminalité

In [20]:
# Ajout d'une tranche d'age dans la démographie

In [1]:
import numpy as np
from models.managers.db_manager import connect_to_bdd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Création des requêtes SQL et des dataframes
conn = connect_to_bdd()

query_demo = "SELECT * FROM demographie"
df_densite = pd.read_sql(query_demo, conn)

query_economie = "SELECT * FROM economie"
df_economie = pd.read_sql(query_economie, conn)

query_securite = "SELECT * FROM securite"
df_securite = pd.read_sql(query_securite, conn)

query_social = "SELECT * FROM social"
df_social = pd.read_sql(query_social, conn)

query_election_t1 = "SELECT * FROM election_2022_t1"
df_election_t1 = pd.read_sql(query_election_t1, conn)

# Fermer la connexion
conn.close()

# Assumons que les connexions et requêtes sont déjà faites et les dataframes sont chargés comme df_densite, df_economie, df_securite, df_social, df_election_t1

# Aggrégation des données
df_densite = df_densite.groupby('code_postal')['dens_pop'].mean().reset_index()
df_securite = df_securite.groupby('code_postal')['tauxpourcent'].mean().reset_index()

# Fusionner les dataframes sur 'code_postal'
df_merged = pd.merge(df_densite, df_economie, on='code_postal', how='inner')
df_merged = pd.merge(df_merged, df_securite, on='code_postal', how='inner')
df_merged = pd.merge(df_merged, df_social, on='code_postal', how='inner')
df_merged = pd.merge(df_merged, df_election_t1, on='code_postal', how='inner')

# Assumons que 'winner' est une colonne dans df_election_t1 qui indique le parti gagnant
# Calculer le ratio entre le niveau de vie médian et le taux de chômage
df_merged['ratio_vie_chomage'] = df_merged['Médiane du niveau de vie 2021'] / df_merged['avg_2022']
df_merged['taux_criminalite'] = df_merged['tauxpourcent']

### Étape 2 : Encodage des caractéristiques catégorielles
label_encoder = LabelEncoder()
for column in df_merged.select_dtypes(include=['object']).columns:
    df_merged[column] = label_encoder.fit_transform(df_merged[column])

### Étape 3 : Préparation des données pour XGBoost
X = df_merged.drop(['winner', 'code_postal'], axis=1)  # Supprimez 'winner' et 'code_postal'
y = label_encoder.fit_transform(df_merged['winner'])

### Étape 4 : Split des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Étape 5 : Configuration et entraînement du modèle XGBoost
model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(set(y)), seed=42)
model.fit(X_train, y_train)

### Étape 6 : Prédiction et évaluation
y_pred = model.predict(X_test)
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

### Étape 7 : Feature importance
importance = model.feature_importances_
features = pd.DataFrame({'Feature': X.columns, 'Importance': importance}).sort_values(by='Importance', ascending=False)
print(features)

### Étape 8 : Optimisation avec GridSearchCV
param_grid = {
    'max_depth': [3, 5, 7]
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=2)
grid_search.fit(X_train, y_train)
print("Meilleurs paramètres: ", grid_search.best_params_)
print("Meilleure précision: {:.2f}%".format(grid_search.best_score_ * 100))

/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_25388/2285983127.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_densite = pd.read_sql(query_demo, conn)
/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_25388/2285983127.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_economie = pd.read_sql(query_economie, conn)
/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_25388/2285983127.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_securite = pd.read_sql(query_securite, conn)
/va

Accuracy: 100.00%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        15
           2       1.00      1.00      1.00      5175
           3       1.00      1.00      1.00     22948
           4       1.00      1.00      1.00        27
           5       1.00      1.00      1.00        26
           6       1.00      1.00      1.00        23
           7       1.00      1.00      1.00         9
           8       1.00      1.00      1.00     38971
           9       1.00      1.00      1.00        52
          10       1.00      1.00      1.00       446

    accuracy                           1.00     67698
   macro avg       1.00      1.00      1.00     67698
weighted avg       1.00      1.00      1.00     67698

[[    6     0     0     0     0     0     0     0     0     0     0]
 [    0    15     0     0     0     0     0     0     0     0     0]
 [    0     0  5175     0     0